In [17]:
import os
import cv2
import argparse
import numpy as np
import pandas as pd
import mediapipe as mp

from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [20]:
# define paths
SOURCE_PATH = "C:/Users/user/Desktop/수어 데이터셋/test"
DEST_PATH = "../data_preprocess/dummy_data"

In [99]:
def norm(npy):  # keypoints normalization
    scaler = MinMaxScaler()
    flat_npy = np.reshape(npy, (-1, np.shape(npy)[-1]))
    scaled_flat_npy = scaler.fit_transform(flat_npy)
    scaled_npy = np.reshape(scaled_flat_npy, np.shape(npy))
    return scaled_npy  

def rm_indexes(npy):
    arr = np.load(npy)
    indices = np.arange(arr.shape[1])
    modified_indices = np.delete(indices, arr.shape[:16])
    return modified_indices

In [100]:
for file in tqdm(os.listdir(SOURCE_PATH)):
    filename = str(os.path.splitext(file)[0])
    filepath = os.path.join(SOURCE_PATH, file)
    cap = cv2.VideoCapture(filepath)

    mp_pose = mp.solutions.pose
    mp_hands = mp.solutions.hands

    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    full_pose_keypoints = []
    full_hands_keypoints = []

    while True:
        opened, image = cap.read()
        if not opened:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hands_results = hands.process(image_rgb)

        if pose_results.pose_landmarks:
            pose_keypoints = []
            for landmark in pose_results.pose_landmarks.landmark:
                pose_keypoints.append([landmark.x, landmark.y, landmark.z])
            full_pose_keypoints.append(pose_keypoints)

        if hands_results.multi_hand_landmarks:
            hands_keypoints = []
            for hand_landmarks in hands_results.multi_hand_landmarks:
                for landmark in hand_landmarks.landmark:
                    hands_keypoints.append([landmark.x, landmark.y, landmark.z])
            full_hands_keypoints.append(hands_keypoints)

    if full_hands_keypoints:
        norm_hands = norm(np.array(full_hands_keypoints))
        np.save(f"{DEST_PATH}/{filename}_hands.npy", norm_hands)

    if full_pose_keypoints:
        norm_pose = norm(np.array(full_pose_keypoints))
        np.save(f"{DEST_PATH}/{filename}_pose.npy", norm_pose)

    cap.release()

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\venv1\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|██████████| 1/1 [00:07<00:00,  7.84s/it]
